In [1]:
from pyspark import SparkConf, SparkContext

from pyspark.sql import SparkSession
import math


In [3]:
def main(f, mobiled, model_dict, x0):
    path_sample = "" % f

    sample = spark.sparkContext.textFile(path_sample).collect()
    
    tp = 0 
    tn = 0 
    fn = 0
    fp = 0
    for line in sample:
        ln = line.replace("(", "")
        ln2 = ln.replace(")", "")
        lnn = ln2.split(",")
        score = x0
        for idx, item in enumerate(lnn):
            if idx == 0:
                label = int(item)
            else:
                if name.startswith("uid_ouid"):
                    continue
                name = str(item.split("=")[0])
                term = str(item.split("=")[1])
                weight = model_dict.get(name+value, 0)
                score += weight
                
        prob = 1.0 / (1+math.exp(-score))
        if prob > 0.5:
            prob_label = 1
        else:
            prob_label = 0
        if label == 1 and prob_label == 1:
            tp += 1
        elif label == 0 and prob_label == 0:
            tn += 1
        elif label == 1 and prob_label == 0:
            fn += 1
        elif label == 0 and prob_label == 1:
            fp += 1
    return tp, tn, fn, fp

        
    
if __name__ == "__main__":
    from datetime import datetime, date, timedelta
    spark = SparkSession.builder.appName("lr_sample_train").getOrCreate()
    mobiled = spark.sql("select mobiletype from songwt.mobileid").collect()
    mobiled_list = [list(mobile)[0] for mobile in mobiled]
    
    path_model = "/user/hdfs/swt/lr/output_0.1/best-model-text/best_model"
    
    model = spark.sparkContext.textFile(path_model).collect()

    model_dict = dict()
    x0 = 0
    for line in model:
        line = line.strip()
        ln = line.split("\t")
        name = ln[0]
        value = ln[1]
        model_dict[name+value] = float(ln[2])
        if name == "(INTERCEPT)":
            x0 = float(ln[2])

    tp_label=0
    tn_label =0
    fp_label=0
    fn_label=0
    for i in range(117, 55, -1):
        da = (date.today() + timedelta(days = -i)).strftime("%Y-%m-%d")
        f = "sample_%s" % str(da)
        
        tp, tn, fn, fp = main(f,mobiled_list, model_dict, x0)
        tp_label += tp
        tn_label += tn
        fp_label += fp
        fn_label += fn
    print(tp_label, tn_label, fp_label, fn_label)
        
        
    
    

598475 6288694 160278 503872
